In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import yfinance as yf
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from datetime import timedelta
from sklearn.decomposition import PCA
import xgboost as xgb

KeyboardInterrupt: 

In [ ]:
fname = ['TSLA','GOOG','JPM','V','CVX','BAC','ABBV','DHR','SCHW','KO','MRK','MCD','TMO','WFC','ABT','COP','BMY','NEE','VZ','LIN','TXN','MS','AMGN','UPS','HON','PM']
tickers = [yf.Ticker(ticker) for ticker in fname]
print(tickers)


In [ ]:
dfs = [] # list for each ticker's dataframe

for ticker in tickers:
    print(ticker)
    # get each financial statement
    pnl = ticker.financials
    bs = ticker.balancesheet
    cf = ticker.cashflow

    # concatenate into one dataframe
    fs = pd.concat([pnl, bs, cf])

    # make dataframe format nicer
    # Swap dates and columns
    data = fs.T
    # reset index (date) into a column
    data = data.reset_index()
    # Rename old index from '' to Date
    data.columns = ['Date', *data.columns[1:]]
    # Add ticker to dataframe
    data['Ticker'] = ticker.ticker
    dfs.append(data)
data.iloc[:,:3]# for display purposes



In [ ]:
data_fund_price = []

for i in range(len(fname)):
    fname_temp = fname[i]
    print(fname_temp)
    
    obj_f = yf.Ticker(fname_temp)
    
    id = fname.index(fname_temp)
    fd_dt = pd.DataFrame(dfs[id])
    
    price = []
    for j in range(len(fd_dt["Date"])):
        strdate = datetime.strptime(str(fd_dt.Date[j]), "%Y-%m-%d %H:%M:%S") #- timedelta(days =1)
        print(strdate)
        enddate = datetime.strptime(str(fd_dt.Date[j]), "%Y-%m-%d %H:%M:%S") + timedelta(days =1)
        print(enddate)
        p_data = obj_f.history(interval='1d', start = strdate, end=enddate)
        print(p_data)
        price.append(p_data["Close"].item())
        
    price_data = pd.DataFrame(price)
    res = pd.concat([fd_dt, price_data], axis=1)
    
    data_fund_price.append(res)

In [ ]:

set0 = set(data_fund_price[0].columns)
for i in range(len(data_fund_price)):
    print(i)
    datablock = data_fund_price[i]
    datablock = datablock.loc[:,~datablock.columns.duplicated()].copy()
    dbnames = set(datablock.columns)
    set0 = set0.intersection(dbnames)

set0
lnames = list(set0)
len(set0)
len(list(set0))
#set0
    

In [ ]:
res0 = pd.DataFrame()
for i in range(len(data_fund_price)):
    print(i)
    datablock = data_fund_price[i]
    datablock = datablock.loc[:,~datablock.columns.duplicated()].copy()
    datablock = datablock[lnames]
    res0 = pd.concat([res0,datablock],axis=0)


In [ ]:
res0.rename({0: "closeP"},axis=1,inplace =True)
res0 = res0.dropna(axis=1)
res0

In [ ]:
X = pd.DataFrame(res0.copy())

Y = X["closeP"]
Y=Y.reset_index()
Y = Y.drop(["index"],axis=1)
X = X.drop(["closeP","Date","Ticker"],axis=1)

X
#Y

In [272]:
scaler = StandardScaler()

scaler.fit(X) 
X_scaled=scaler.transform(X)

pca = PCA(n_components= 3)
pca.fit(X_scaled)
X_pca = pca.transform(X_scaled)
print("shape of X_pca", X_pca.shape)

ex_variance=np.var(X_pca,axis=0)
ex_variance_ratio = ex_variance/np.sum(ex_variance)
print(ex_variance_ratio)

shape of X_pca (104, 3)
[0.6480902  0.2613954  0.09051441]


In [289]:
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from numpy import absolute
from numpy import sqrt
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from statistics import mean
import random
random.seed(10)

RMSE_fin = []

for i in range(1,20):
    pca=PCA(n_components=i)
    PC=pca.fit_transform(X)
    principalDF=pd.DataFrame(data=PC)
    finalDf = pd.concat([principalDF, Y], axis = 1)
    Xfinal = principalDF
    yfinal = Y
    RMSE =[]
    
    #define cross-validation method to use
    cv = KFold(n_splits=10, random_state=1, shuffle=True) 

    #build multiple linear regression model
    model = LinearRegression()

    #
    scores = cross_val_score(model, Xfinal, yfinal, scoring='neg_mean_squared_error',
                         cv=cv, n_jobs=-1)

    #view RMSE
    rmse = sqrt(mean(absolute(scores)))
        
    RMSE_fin.append(rmse)
    
print(RMSE_fin)
    
